In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle, bz2
import datetime
from collections import OrderedDict
import pylogit as pl
from scipy import stats
import ast
from itertools import permutations
from sympy import symbols, solve
import random
random.seed(42)
np.random.seed(42)

In [2]:
custom_alt_id = 'mode_id'
obs_id_column = 'Custom_Choice_id'
choice_column = 'Choice'

In [3]:
try2 = pd.read_csv('../forLogit0701.csv').drop('Unnamed: 0',axis=1)
try4 = try2.groupby(['initial_adl_time_x','CARRIER','ARR_LOCID']).count()[['hub']].reset_index()
try4 = try4[try4['hub']>1]

In [4]:
len(try4)

2935

In [5]:
Custom_Choice_id = 0
convex = 1.1
for_logit = []
for i in range(len(try4)):
    try3 = try2[(try2['initial_adl_time_x']==try4.iloc[i]['initial_adl_time_x'])&(try2['ARR_LOCID']==try4.iloc[i]['ARR_LOCID'])&(try2['CARRIER']==try4.iloc[i]['CARRIER'])]
    try3 = try3.sort_values('EDCT_ON')
    for q in range(len(try3)-1):#q+1 th slot
        try9 = try3.tail(len(try3)-q)
        try9 = try9.sort_values('OAG_S_ARR')
        slot_time = np.sort(list(try9['EDCT_ON']))#sorted
        first_slot_time = slot_time[0]
        slot_time = slot_time[1:]
        sch_time = list(try9['OAG_S_ARR'])#sorted
        
        Custom_Choice_id = Custom_Choice_id + 1
        for j in range(len(try3)-q):
            
            if try9.iloc[j]['min_EDCT_on']<=first_slot_time:
                sch_time_new = sch_time[:j] + sch_time[j+1:]
                delay_convex = 0
                for p in range(len(try3)-q-1):
                    delay_convex = delay_convex + (max(0,-sch_time_new[p]+slot_time[p])/3600)**convex
                if try9.iloc[j]['EDCT_ON']==first_slot_time:
                    latent = int(try9.iloc[j]['OAG_S_ARR']<slot_time[0] - 2*3600)*int(try9.iloc[j]['OAG_S_ARR']>first_slot_time - 1*3600)
                    for_logit.append([Custom_Choice_id,1,j,delay_convex,(max(0,-sch_time[j]+list(try9['EDCT_ON'])[0])/3600)**convex,try9.iloc[j]['initial_adl_time_x'],
                                     try9.iloc[j]['hub'],try9.iloc[j]['Pass_on'],try9.iloc[j]['Seats'],try9.iloc[j]['avg_fare'],try9.iloc[j]['connect'],try9.iloc[j]['buff_time'],try9.iloc[j]['oversea'],try9.iloc[j]['#inter'],try9.iloc[j]['dep_delta'],1,latent])
                else:
                    latent = int(try9.iloc[j]['OAG_S_ARR']<slot_time[0] - 2*3600)*int(try9.iloc[j]['OAG_S_ARR']>first_slot_time - 1*3600)
                    for_logit.append([Custom_Choice_id,0,j,delay_convex,(max(0,-sch_time[j]+list(try9['EDCT_ON'])[0])/3600)**convex,try9.iloc[j]['initial_adl_time_x'],
                                     try9.iloc[j]['hub'],try9.iloc[j]['Pass_on'],try9.iloc[j]['Seats'],try9.iloc[j]['avg_fare'],try9.iloc[j]['connect'],try9.iloc[j]['buff_time'],try9.iloc[j]['oversea'],try9.iloc[j]['#inter'],try9.iloc[j]['dep_delta'],1,latent])

In [6]:
forLogit = pd.DataFrame(for_logit,columns = ['Custom_Choice_id','Choice','swap','delayOthers','delayIt','GDP_time',
                                            'hub','Pass_on','Seats','avg_fare','connect','buff_time','oversea','numberInter','dep_delta','mode_id','latent'])

temp = forLogit.groupby('Custom_Choice_id').sum()
temp = temp[temp['Choice']==1].reset_index()
temp = list(temp['Custom_Choice_id'])
forLogit = forLogit[forLogit['Custom_Choice_id'].isin(temp)]


temp = forLogit.groupby('Custom_Choice_id').count()
temp = temp[temp['Choice']>=2].reset_index()
temp = list(temp['Custom_Choice_id'])
temp = random.sample(temp, 20000)
forLogit = forLogit[forLogit['Custom_Choice_id'].isin(temp)]

forLogit

/var/folders/pd/42djns7x589bw787p86syckm0000gn/T/ipykernel_82405/175538299.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp = forLogit.groupby('Custom_Choice_id').sum()


,Custom_Choice_id,Choice,swap,delayOthers,delayIt,GDP_time,hub,Pass_on,Seats,avg_fare,connect,buff_time,oversea,numberInter,dep_delta,mode_id,latent
1,2,1,0,2.533144,0.605056,2019-01-01 15:47:00,True,118.333790,160.0,215.074737,True,4920.0,0,14,9780.0,1,0
2,2,0,1,3.757209,0.000000,2019-01-01 15:47:00,True,134.803060,160.0,262.278235,True,3720.0,0,28,3360.0,1,0
3,2,0,2,3.991873,0.000000,2019-01-01 15:47:00,True,148.379468,160.0,207.820309,True,16140.0,0,30,10380.0,1,0
7,4,0,0,1.022067,1.184790,2019-01-01 15:47:00,True,148.379468,160.0,207.820309,True,16140.0,0,30,10380.0,1,0
8,4,1,1,1.478126,0.728731,2019-01-01 15:47:00,True,142.526316,181.0,234.620909,True,3300.0,0,29,10380.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421589,53883,0,3,4.100994,0.000000,2019-12-31 11:23:00,False,124.193651,142.0,210.206596,True,4260.0,0,8,9420.0,1,0
421590,53883,0,4,4.136375,0.000000,2019-12-31 11:23:00,False,131.648641,154.0,212.489781,True,-180.0,0,8,7260.0,1,0
421595,53885,0,0,2.375752,0.185939,2019-12-31 11:23:00,True,144.810411,154.0,255.202581,True,-4080.0,1,8,7560.0,1,0
421596,53885,0,1,2.422365,0.139326,2019-12-31 11:23:00,False,124.193651,142.0,210.206596,True,4260.0,0,8,9420.0,1,0


In [7]:
forLogit['Connect'] = forLogit.apply(lambda row: int(row.connect),axis=1)
forLogit['Hub'] = forLogit.apply(lambda row: int(row.hub),axis=1)
forLogit['Revenue'] = forLogit.apply(lambda row: row.avg_fare*row.Pass_on,axis=1)

In [8]:
forLogit['SeatInter'] = forLogit.apply(lambda row: row.Seats*row.numberInter,axis=1)

In [9]:
custom_alt_id = 'mode_id'
obs_id_column = 'Custom_Choice_id'
choice_column = 'Choice'

basic_specification = OrderedDict()
basic_names = OrderedDict()

# basic_specification["delayIt"] = [1]
# basic_names["delayIt"] = ['delayIt']

basic_specification["delayOthers"] = [1]
basic_names["delayOthers"] = ['delayOthers']

# basic_specification["swap"] = [1]
# basic_names["swap"] = ['swap']

mnl = pl.create_choice_model(data=forLogit,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

mnl.fit_mle(np.zeros(1))
mnl.get_statsmodels_summary()

Log-likelihood at zero: -35,705.3612
Initial Log-likelihood: -35,705.3612
Estimation Time for Point Estimation: 0.04 seconds.
Final log-likelihood: -32,746.4118


/Users/jing/anaconda3/lib/python3.10/site-packages/pylogit/estimation.py:678: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      Choice   No. Observations:               20,000
Model:             Multinomial Logit Model   Df Residuals:                   19,999
Method:                                MLE   Df Model:                            1
Date:                     Wed, 10 Jul 2024   Pseudo R-squ.:                   0.083
Time:                             19:54:41   Pseudo R-bar-squ.:               0.083
AIC:                            65,494.824   Log-Likelihood:            -32,746.412
BIC:                            65,502.727   LL-Null:                   -35,705.361
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
delayOthers    -0.5461      0.008    -71.058      0.000      -0.561      -0.531
===============================================================================
"""

In [10]:
custom_alt_id = 'mode_id'
obs_id_column = 'Custom_Choice_id'
choice_column = 'Choice'

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["delayOthers"] = [1]
basic_names["delayOthers"] = ['delayOthers']

basic_specification["swap"] = [1]
basic_names["swap"] = ['swap']


mnl = pl.create_choice_model(data=forLogit,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

mnl.fit_mle(np.zeros(2))
mnl.get_statsmodels_summary()

Log-likelihood at zero: -35,705.3612
Initial Log-likelihood: -35,705.3612
Estimation Time for Point Estimation: 0.09 seconds.
Final log-likelihood: -30,633.6027


/Users/jing/anaconda3/lib/python3.10/site-packages/pylogit/estimation.py:678: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      Choice   No. Observations:               20,000
Model:             Multinomial Logit Model   Df Residuals:                   19,998
Method:                                MLE   Df Model:                            2
Date:                     Wed, 10 Jul 2024   Pseudo R-squ.:                   0.142
Time:                             19:54:50   Pseudo R-bar-squ.:               0.142
AIC:                            61,271.205   Log-Likelihood:            -30,633.603
BIC:                            61,287.012   LL-Null:                   -35,705.361
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
delayOthers    -0.0637      0.010     -6.238      0.000      -0.084      -0.044
swap           -0.2957      0.005    -54.923      0.000      -0.306      -0.285
===============================================================================
"""

In [11]:
custom_alt_id = 'mode_id'
obs_id_column = 'Custom_Choice_id'
choice_column = 'Choice'

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["delayOthers"] = [1]
basic_names["delayOthers"] = ['delayOthers']

basic_specification["swap"] = [1]
basic_names["swap"] = ['swap']

basic_specification["delayIt"] = [1]
basic_names["delayIt"] = ['delayIt']

mnl = pl.create_choice_model(data=forLogit,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

mnl.fit_mle(np.zeros(3))
mnl.get_statsmodels_summary()

Log-likelihood at zero: -35,705.3612
Initial Log-likelihood: -35,705.3612


/Users/jing/anaconda3/lib/python3.10/site-packages/pylogit/estimation.py:678: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 0.42 seconds.
Final log-likelihood: -26,408.3233


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      Choice   No. Observations:               20,000
Model:             Multinomial Logit Model   Df Residuals:                   19,997
Method:                                MLE   Df Model:                            3
Date:                     Wed, 10 Jul 2024   Pseudo R-squ.:                   0.260
Time:                             19:55:00   Pseudo R-bar-squ.:               0.260
AIC:                            52,822.647   Log-Likelihood:            -26,408.323
BIC:                            52,846.357   LL-Null:                   -35,705.361
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
delayOthers    -6.6049      0.093    -70.697      0.000      -6.788      -6.422
swap           -0.2980      0.004    -77.194      0.000      -0.306      -0.290
delayIt        -6.7412      0.094    -71.864      0.000      -6.925      -6.557
===============================================================================
"""

In [17]:
forLogit['test1'] = forLogit['delayIt']*100

In [18]:
custom_alt_id = 'mode_id'
obs_id_column = 'Custom_Choice_id'
choice_column = 'Choice'

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["delayOthers"] = [1]
basic_names["delayOthers"] = ['delayOthers']

basic_specification["swap"] = [1]
basic_names["swap"] = ['swap']

basic_specification["test1"] = [1]
basic_names["test1"] = ['test1']

mnl = pl.create_choice_model(data=forLogit,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

mnl.fit_mle(np.zeros(3))
mnl.get_statsmodels_summary()

Log-likelihood at zero: -35,705.3612
Initial Log-likelihood: -35,705.3612


/Users/jing/anaconda3/lib/python3.10/site-packages/pylogit/estimation.py:678: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 0.43 seconds.
Final log-likelihood: -26,500.7510


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      Choice   No. Observations:               20,000
Model:             Multinomial Logit Model   Df Residuals:                   19,997
Method:                                MLE   Df Model:                            3
Date:                     Wed, 10 Jul 2024   Pseudo R-squ.:                   0.258
Time:                             19:59:28   Pseudo R-bar-squ.:               0.258
AIC:                            53,007.502   Log-Likelihood:            -26,500.751
BIC:                            53,031.212   LL-Null:                   -35,705.361
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
delayOthers    -6.0259      0.093    -64.538      0.000      -6.209      -5.843
swap           -0.3512      0.004    -88.026      0.000      -0.359      -0.343
test1          -0.0619      0.001    -66.085      0.000      -0.064      -0.060
===============================================================================
"""

In [12]:
forLogittt = forLogit[['swap', 'delayOthers', 'delayIt']]

In [13]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(forLogittt)

# Apply PCA
pca = PCA()
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame with the principal components
principal_df = pd.DataFrame(data = principal_components, columns = ['PC1', 'PC2', 'PC3'])

principal_df

,PC1,PC2,PC3
0,-1.043329,0.031353,-0.087776
1,-1.041872,-0.245506,-0.138570
2,-0.978337,-0.288876,-0.081738
3,-0.990674,0.254535,0.021908
4,-0.980287,0.034677,0.003730
...,...,...,...
159531,-0.916325,-0.332365,-0.023443
159532,-0.855206,-0.375925,0.035711
159533,-1.096545,-0.131181,-0.152503
159534,-1.040996,-0.192790,-0.100883


In [14]:
forLogit['one'] = list(principal_df['PC1'])
forLogit['two'] = list(principal_df['PC2'])
forLogit['three'] = list(principal_df['PC3'])

In [15]:
forLogit

,Custom_Choice_id,Choice,swap,delayOthers,delayIt,GDP_time,hub,Pass_on,Seats,avg_fare,...,dep_delta,mode_id,latent,Connect,Hub,Revenue,SeatInter,one,two,three
1,2,1,0,2.533144,0.605056,2019-01-01 15:47:00,True,118.333790,160.0,215.074737,...,9780.0,1,0,1,1,25450.608761,2240.0,-1.043329,0.031353,-0.087776
2,2,0,1,3.757209,0.000000,2019-01-01 15:47:00,True,134.803060,160.0,262.278235,...,3360.0,1,0,1,1,35355.908783,4480.0,-1.041872,-0.245506,-0.138570
3,2,0,2,3.991873,0.000000,2019-01-01 15:47:00,True,148.379468,160.0,207.820309,...,10380.0,1,0,1,1,30836.266906,4800.0,-0.978337,-0.288876,-0.081738
7,4,0,0,1.022067,1.184790,2019-01-01 15:47:00,True,148.379468,160.0,207.820309,...,10380.0,1,0,1,1,30836.266906,4800.0,-0.990674,0.254535,0.021908
8,4,1,1,1.478126,0.728731,2019-01-01 15:47:00,True,142.526316,181.0,234.620909,...,10380.0,1,0,1,1,33439.653780,5249.0,-0.980287,0.034677,0.003730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421589,53883,0,3,4.100994,0.000000,2019-12-31 11:23:00,False,124.193651,142.0,210.206596,...,9420.0,1,0,1,0,26106.324546,1136.0,-0.916325,-0.332365,-0.023443
421590,53883,0,4,4.136375,0.000000,2019-12-31 11:23:00,False,131.648641,154.0,212.489781,...,7260.0,1,0,1,0,27973.990914,1232.0,-0.855206,-0.375925,0.035711
421595,53885,0,0,2.375752,0.185939,2019-12-31 11:23:00,True,144.810411,154.0,255.202581,...,7560.0,1,0,1,1,36955.990708,1232.0,-1.096545,-0.131181,-0.152503
421596,53885,0,1,2.422365,0.139326,2019-12-31 11:23:00,False,124.193651,142.0,210.206596,...,9420.0,1,0,1,0,26106.324546,1136.0,-1.040996,-0.192790,-0.100883


In [16]:
custom_alt_id = 'mode_id'
obs_id_column = 'Custom_Choice_id'
choice_column = 'Choice'

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["one"] = [1]
basic_names["one"] = ['one']

basic_specification["two"] = [1]
basic_names["two"] = ['two']

basic_specification["three"] = [1]
basic_names["three"] = ['three']

mnl = pl.create_choice_model(data=forLogit,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

mnl.fit_mle(np.zeros(3))
mnl.get_statsmodels_summary()

Log-likelihood at zero: -35,705.3612
Initial Log-likelihood: -35,705.3612


/Users/jing/anaconda3/lib/python3.10/site-packages/pylogit/estimation.py:678: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 0.43 seconds.
Final log-likelihood: -26,431.5579


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      Choice   No. Observations:               20,000
Model:             Multinomial Logit Model   Df Residuals:                   19,997
Method:                                MLE   Df Model:                            3
Date:                     Wed, 10 Jul 2024   Pseudo R-squ.:                   0.260
Time:                             19:57:39   Pseudo R-bar-squ.:               0.260
AIC:                            52,869.116   Log-Likelihood:            -26,431.558
BIC:                            52,892.826   LL-Null:                   -35,705.361
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
one         -326.1762      4.738    -68.839      0.000    -335.463    -316.889
two          -38.9063      0.590    -65.922      0.000     -40.063     -37.750
three        298.2467      4.405     67.713      0.000     289.614     306.879
==============================================================================
"""

In [19]:
forLogit['test2'] = forLogit['two']*100

In [20]:
custom_alt_id = 'mode_id'
obs_id_column = 'Custom_Choice_id'
choice_column = 'Choice'

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["one"] = [1]
basic_names["one"] = ['one']

basic_specification["test2"] = [1]
basic_names["test2"] = ['test2']

basic_specification["three"] = [1]
basic_names["three"] = ['three']

mnl = pl.create_choice_model(data=forLogit,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

mnl.fit_mle(np.zeros(3))
mnl.get_statsmodels_summary()

Log-likelihood at zero: -35,705.3612
Initial Log-likelihood: -35,705.3612


/Users/jing/anaconda3/lib/python3.10/site-packages/pylogit/estimation.py:678: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  results = minimize(estimator.calc_neg_log_likelihood_and_neg_gradient,


Estimation Time for Point Estimation: 0.35 seconds.
Final log-likelihood: -26,400.8448


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      Choice   No. Observations:               20,000
Model:             Multinomial Logit Model   Df Residuals:                   19,997
Method:                                MLE   Df Model:                            3
Date:                     Wed, 10 Jul 2024   Pseudo R-squ.:                   0.261
Time:                             19:59:55   Pseudo R-bar-squ.:               0.261
AIC:                            52,807.690   Log-Likelihood:            -26,400.845
BIC:                            52,831.400   LL-Null:                   -35,705.361
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
one         -333.2515      4.741    -70.294      0.000    -342.543    -323.960
test2         -0.3979      0.006    -67.410      0.000      -0.409      -0.386
three        305.3000      4.407     69.271      0.000     296.662     313.938
==============================================================================
"""